# Images to TFRecords

I usually traffic in large datasets that can't fit in memory.  The TFRecords format is a common solution to that.

REFERENCE SOURCE: https://www.kaggle.com/code/kawaitsoi/convert-images-and-masks-to-tfrecords-file - by user kawaitsoi

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tif
import pandas as pd
from loguru import logger

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('../data/folds/foldSet_1.csv', index_col=0)

df.info()
df.head(3)

<class 'pandas.core.frame.DataFrame'>
Index: 612 entries, 0 to 611
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   filename         612 non-null    object
 1   frame_number     612 non-null    int64 
 2   sequence         612 non-null    int64 
 3   img_filepath     612 non-null    object
 4   gt_filepath      612 non-null    object
 5   crossfold_group  612 non-null    int64 
 6   fold_number      612 non-null    int64 
 7   dset             612 non-null    object
dtypes: int64(4), object(4)
memory usage: 43.0+ KB


,filename,frame_number,sequence,img_filepath,gt_filepath,crossfold_group,fold_number,dset
0,1.tif,1,1,D:/Downloads/Datasets/archive/CVC-ClinicDB/Ori...,D:/Downloads/Datasets/archive/CVC-ClinicDB/Gro...,1,1,validation
1,2.tif,2,1,D:/Downloads/Datasets/archive/CVC-ClinicDB/Ori...,D:/Downloads/Datasets/archive/CVC-ClinicDB/Gro...,10,1,training
2,3.tif,3,1,D:/Downloads/Datasets/archive/CVC-ClinicDB/Ori...,D:/Downloads/Datasets/archive/CVC-ClinicDB/Gro...,3,1,test


In [3]:
df['crossfold_group'].value_counts() 

crossfold_group
1     74
2     71
3     68
5     67
4     67
6     56
7     54
8     53
9     52
10    50
Name: count, dtype: int64

In [4]:
df['dset'].value_counts() 

dset
training      403
validation    141
test           68
Name: count, dtype: int64

In [5]:
train = df[df['dset'] == 'training'].reset_index(drop=True)
val = df[df['dset'] == 'validation'].reset_index(drop=True)
tst = df[df['dset'] == 'test'].reset_index(drop=True)

## create

In [6]:
# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _bytes_feature(value):
  """
  Returns a bytes_list from a string / byte.
  SOURCE: https://www.tensorflow.org/tutorials/load_data/tfrecord
  """
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """
  Returns a float_list from a float / double.
  SOURCE: https://www.tensorflow.org/tutorials/load_data/tfrecord
  """
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """
  Returns an int64_list from a bool / enum / int / uint.
  SOURCE: https://www.tensorflow.org/tutorials/load_data/tfrecord
  """
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def to_tfrecords(dframe, img_read_func, filestem):
    """
    """
    #construct list for image paths and their corresponding masks in the same order
    #image_path = "../data/train/*.jpg"
    image_path = dframe['img_filepath']
    mask_path = dframe['gt_filepath']
    
    # path to where the tfrecords will be stored (change to your customized path).
    # I can't run this here in Kaggle because of permission. 
    # Any comment on how to write temporary files in Kaggle will be appreciated.
    tfrecords_filename = f'../data/tfrecords/{filestem}.tfrecords'
    
    # get a writer for the tfrecord file.
    with tf.io.TFRecordWriter(tfrecords_filename) as writer:
        # write data/masks into tfrecords
        for i in range(len(image_path)):
            img = np.array(img_read_func(image_path[i]))
            mask = np.array(img_read_func(mask_path[i]))
        
            height, width = img.shape[0], img.shape[1]
 
            img_raw = img.tostring()
            mask_raw = mask.tostring()
            
            # save the heights and widths as well so, which 
            # are needed when decoding from tfrecords back to images
            example = tf.train.Example(features=tf.train.Features(feature={
                                                                  'height': _int64_feature(height),
                                                                  'width': _int64_feature(width),
                                                                  'image_raw': _bytes_feature(img_raw),
                                                                  'mask_raw': _bytes_feature(mask_raw)}
                                                                  ))
            writer.write(example.SerializeToString())

    print(f'Wrote file to: {tfrecords_filename}')

    return tfrecords_filename

In [ ]:
tfrecords_filename = to_tfrecords(train, tif.imread, 'foldSet1_TRAIN')

## Validate/Read

In [ ]:
def read_record(string_record: str):
    example = tf.train.Example()
    example.ParseFromString(string_record)
    
    height = int(example.features.feature['height'].int64_list.value[0])
    width = int(example.features.feature['width'].int64_list.value[0])
    
    img_string = (example.features.feature['image_raw'].bytes_list.value[0])
    mask_string = (example.features.feature['mask_raw'].bytes_list.value[0])
    
    img_1d = np.fromstring(img_string, dtype=np.uint8)
    mask_1d = np.fromstring(mask_string, dtype=np.uint8)
    
    #reshape back to their original shape from a 1D array read from tfrecords
    img = img_1d.reshape((height, width, -1))
    mask = mask_1d.reshape((height, width))
    return img, mask

In [ ]:
#run the following to verify the created tfrecord file.
record_iterator = tf.compat.v1.io.tf_record_iterator(path=tfrecords_filename)

for string_record in record_iterator:
    example = tf.train.Example()
    example.ParseFromString(string_record)
    
    height = int(example.features.feature['height'].int64_list.value[0])
    width = int(example.features.feature['width'].int64_list.value[0])
    
    img_string = (example.features.feature['image_raw'].bytes_list.value[0])
    mask_string = (example.features.feature['mask_raw'].bytes_list.value[0])
    
    img_1d = np.fromstring(img_string, dtype=np.uint8)
    mask_1d = np.fromstring(mask_string, dtype=np.uint8)
    
    #reshape back to their original shape from a 1D array read from tfrecords
    img = img_1d.reshape((height, width, -1))
    mask = mask_1d.reshape((height, width))
    
    plt.imshow(img)
    plt.show()
    plt.imshow(mask)
    plt.show()